# Training a Logistic Regression Model

## Imports

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold


from joblib import dump

## Find the git root Directory

In [2]:
# Get the current working directory
cwd = Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

# Seed and Generator for Reproducibility

In [3]:
rnd_val = 0  # Random value for all seeds
rng = np.random.default_rng(seed=rnd_val)  # random number generator

## Create paths

In [4]:
filename1 = "Plate_1_sc_norm_fs.parquet"
plate_path = Path(
    f"{root_dir}/nf1_painting_repo/3.processing_features/data/feature_selected_data"
)

path1 = plate_path / filename1

data_path = Path("data")

# Create the parent directories if they don't exist
data_path.mkdir(parents=True, exist_ok=True)

## Generate plate dataframe

In [5]:
# Returns the dataframe generated by the csv
platedf = pd.read_parquet(path1)

# Preprocess Data

## Create Classes

In [6]:
target_column = "Metadata_genotype"
stratify_column = "Metadata_Well"

# These represent the fractions of the entire dataset
train_val_frac = 0.85
test_frac = 1 - train_val_frac

## Down-sample and stratify by well

In [7]:
smallest_gene = platedf[target_column].value_counts().min()
platedata = pd.DataFrame()

for gene in platedf[target_column].unique():
    df = platedf.loc[platedf["Metadata_genotype"] == gene]
    df_frac = smallest_gene / len(df)
    stratwell = df.groupby(stratify_column, group_keys=False).apply(
        lambda x: x.sample(frac=df_frac, random_state=rnd_val)
    )
    platedata = pd.concat([platedata, stratwell], axis="rows")

## Stratified Train-test split

In [8]:
traindf, testdf = train_test_split(
    platedata,
    train_size=train_val_frac,
    stratify=platedata[[target_column, stratify_column]],
    shuffle=True,
    random_state=rnd_val,
)

## Encode Labels

In [9]:
le = LabelEncoder()
testdf["label"] = le.fit_transform(testdf[target_column].values)
traindf["label"] = le.transform(traindf[target_column].values)

/tmp/ipykernel_30430/4215398058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["label"] = le.fit_transform(testdf[target_column].values)
/tmp/ipykernel_30430/4215398058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindf["label"] = le.transform(traindf[target_column].values)


## Remove unecessary columns

In [10]:
# Remove Metadata
feat_col = [col for col in platedata.columns if "Metadata" not in col]

# Keep the label column
feat_col.append("label")

traindf = traindf[feat_col]
testdf = testdf[feat_col]

# Model Training

In [11]:
# Default number of splits
num_splits = 5


def kcross_val(model, X_data, y_data, splits=num_splits):
    """
    Parameters
    ----------
    model: sklearn model
        The model to be trained and evaluated.
    X_data : Pandas Dataframe
        The preprocessed dataframe with features.
    y_data : Pandas Dataframe
        The preprocessed dataframe with labels.
    splits: int
        The number of splits for k cross validation

    Returns
    -------
    best_model: Sklearn Model
        The best model based on the validation accuracy
    """

    cv = KFold(n_splits=splits, random_state=rnd_val, shuffle=True)
    max_acc = 0  # Keep track of the maximum accuracy

    # iterates through index splits
    for train_idx, val_idx in cv.split(X=X_data, y=y_data):
        # Validation labels converted to ints
        yval = y_data.iloc[val_idx].astype(np.int64)

        # Training labels converted to ints
        ytrain = y_data.iloc[train_idx].astype(np.int64)

        # Validation features
        Xval = X_data.iloc[val_idx]

        # Training features
        Xtrain = X_data.iloc[train_idx]

        # Fitting the model
        model.fit(Xtrain, ytrain)

        # Scoring the model on the validation set
        acc = model.score(Xval, yval)

        # Find the model with the best validation accuracy
        if max_acc < acc:
            best_model = model

    return best_model

In [12]:
lr = LogisticRegression(max_iter=1000, solver="sag", random_state=rnd_val, n_jobs=-1)

kcross_val(lr, X_data=traindf.drop("label", axis="columns"), y_data=traindf["label"])

/home/camo/anaconda3/envs/nf1_analysis/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(max_iter=1000, n_jobs=-1, random_state=0, solver='sag')

## Save Data

In [13]:
dump(lr, data_path / "lr_model.joblib")
dump(testdf, data_path / "testdf.joblib")
dump(le, data_path / "label_encoder.joblib")

['data/label_encoder.joblib']